## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Ushuaia,-54.8000,-68.3000,55.69,35,24,2.82,AR,2021-11-27 16:01:19,few clouds
1,1,Albany,42.6001,-73.9662,27.25,59,12,15.75,US,2021-11-27 16:01:58,few clouds
2,2,Iqaluit,63.7506,-68.5145,20.75,88,100,8.43,CA,2021-11-27 16:02:03,light snow
3,3,Ribeira Grande,38.5167,-28.7000,66.60,55,20,17.27,PT,2021-11-27 16:03:17,few clouds
4,4,Acari,-15.4311,-74.6158,74.89,66,82,8.99,PE,2021-11-27 16:03:18,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 60


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                   (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Ushuaia,-54.8000,-68.3000,55.69,35,24,2.82,AR,2021-11-27 16:01:19,few clouds
24,24,Punta Arenas,-53.1500,-70.9167,53.71,40,75,19.57,CL,2021-11-27 16:03:23,broken clouds
35,35,Kaeo,-35.1000,173.7833,55.76,91,41,2.19,NZ,2021-11-27 16:03:26,scattered clouds
79,79,Canon City,38.4410,-105.2424,50.92,49,1,0.00,US,2021-11-27 16:04:38,clear sky
94,94,Pacific Grove,36.6177,-121.9166,51.91,50,1,0.00,US,2021-11-27 16:05:42,clear sky
97,97,Santa Lucia,27.9117,-15.5407,58.32,68,40,20.71,ES,2021-11-27 16:05:43,scattered clouds
103,103,Lebu,-37.6167,-73.6500,58.19,67,1,15.64,CL,2021-11-27 16:05:45,clear sky
113,113,Avera,33.1940,-82.5271,51.31,33,1,3.85,US,2021-11-27 16:05:47,clear sky
130,130,Portland,45.5234,-122.6762,52.39,87,90,6.91,US,2021-11-27 16:03:03,overcast clouds
143,143,Duki,30.1541,68.5728,56.71,20,0,4.25,PK,2021-11-27 16:06:56,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.isnull().sum()

City ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
Current Description    0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ushuaia,AR,55.69,few clouds,-54.8000,-68.3000,
24,Punta Arenas,CL,53.71,broken clouds,-53.1500,-70.9167,
35,Kaeo,NZ,55.76,scattered clouds,-35.1000,173.7833,
79,Canon City,US,50.92,clear sky,38.4410,-105.2424,
94,Pacific Grove,US,51.91,clear sky,36.6177,-121.9166,
97,Santa Lucia,ES,58.32,scattered clouds,27.9117,-15.5407,
103,Lebu,CL,58.19,clear sky,-37.6167,-73.6500,
113,Avera,US,51.31,clear sky,33.1940,-82.5271,
130,Portland,US,52.39,overcast clouds,45.5234,-122.6762,
143,Duki,PK,56.71,clear sky,30.1541,68.5728,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat}, {lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == ""].index, inplace=True)
hotel_df.count()

City                   63
Country                63
Max Temp               63
Current Description    63
Lat                    63
Lng                    63
Hotel Name             63
dtype: int64

In [33]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp}°F </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))